In [1]:
%classpath add mvn org.apache.spark spark-sql_2.11 2.1.0
org.apache.log4j.Logger.getRootLogger().setLevel(org.apache.log4j.Level.ERROR);


null

In [2]:
import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._ 


import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._


In [3]:
val spark = SparkSession.builder() 
  .master("local[*]") 
  .config("spark.executor.memory", "3g")
  .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse")
  .appName("NohupReader")
  .getOrCreate()


org.apache.spark.sql.SparkSession@24d840d6

In [4]:
import spark.implicits._

val research_home: String = scala.util.Properties.envOrElse("RESEARCH_HOME", "/home/acald013/Research/")
val folder = s"${research_home}Scripts/Misc/"
val prefix = "nohup"

val nohup = spark.read.textFile(s"${folder}${prefix}.out")


org.apache.spark.sql.SparkSession$implicits$@61a6ee71

In [5]:
println(nohup.count)

72


null

In [13]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)
case class Line(line: String, n: Long)
case class Run(runID: Long, method: String, epsilon: Double, mu: Int, delta: Int)

implicit class DatasetOps(ds: org.apache.spark.sql.Dataset[_]) {
    def display(rows: Int = 20) = {
        import com.twosigma.beakerx.scala.table.TableDisplay
        val columns = ds.columns
        val rowVals = ds.toDF.take(rows)
        val t = new TableDisplay(rowVals map (row => (columns zip row.toSeq).toMap))
        t.display()
    }
}

defined class Line
defined class Run
defined class DatasetOps


In [8]:
val lines = nohup.toDF("line").withColumn("n", monotonicallyIncreasingId).as[Line].cache()
val nLines = lines.count()
lines.show(10, truncate = false)

+------------------------------------------------------------------------------------------------------------------------+---+
|line                                                                                                                    |n  |
+------------------------------------------------------------------------------------------------------------------------+---+
|FLOCKFINDER=bfe;TIME=Sun Jun  3 18:06:34 PDT 2018;RUN=1528074394;EPSILON=40;MU=4;DELTA=5;SCRIPT=BfeBenchmark;EVENT=Start|0  |
|                                                                                                                        |1  |
|real	0m45.795s                                                                                                          |2  |
|user	0m45.618s                                                                                                          |3  |
|sys	0m0.177s                                                                                                  

null

In [11]:
val indicesRun = lines.filter{ l => 
        l.line.contains("EVENT=Start") || l.line.contains("EVENT=End")
    }
    .orderBy("n")
    .select("n")
    .collect()
    .toList
    .map(_.getLong(0))
    .grouped(2)
    .toList
    .map(pair => (pair.head, pair.last))
    .filter(r => r._1 != r._2)
    .zipWithIndex
val indexRun = spark.createDataset(indicesRun)
    .flatMap{ pair => 
        (pair._1._1 to pair._1._2)
        .toList.map(v => (pair._2, v))
    }
    .toDF("runID","n")
    .cache
indexRun.show(20, truncate = false)

+-----+---+
|runID|n  |
+-----+---+
|0    |0  |
|0    |1  |
|0    |2  |
|0    |3  |
|0    |4  |
|0    |5  |
|0    |6  |
|0    |7  |
|1    |8  |
|1    |9  |
|1    |10 |
|1    |11 |
|1    |12 |
|1    |13 |
|1    |14 |
|1    |15 |
|2    |16 |
|2    |17 |
|2    |18 |
|2    |19 |
+-----+---+
only showing top 20 rows



null

In [14]:
val runs = indexRun.join(lines, "n").
    groupBy("runID").
    agg(max($"n").alias("n")).
    join(lines, "n").
    select("runID", "line").
    orderBy("runID").
    map{ row =>
        val runID = row.getInt(0)
        val line  = row.getString(1)
        var arr  = line.split(";")
        val method  = arr(0).split("=")(1)
        val epsilon = arr(3).split("=")(1).toDouble
        val mu      = arr(4).split("=")(1).toInt
        val delta   = arr(5).split("=")(1).toInt
        Run(runID, method, epsilon, mu, delta)
    }.
    cache
val nRuns = runs.count()
runs.show(nRuns.toInt, truncate = false)

+-----+------+-------+---+-----+
|runID|method|epsilon|mu |delta|
+-----+------+-------+---+-----+
|0    |bfe   |40.0   |4  |5    |
|1    |bfe   |50.0   |4  |5    |
|2    |bfe   |60.0   |4  |5    |
|3    |bfe   |70.0   |4  |5    |
|4    |bfe   |80.0   |4  |5    |
|5    |bfe   |90.0   |4  |5    |
|6    |bfe   |100.0  |4  |5    |
|7    |bfe   |150.0  |4  |5    |
|8    |bfe   |200.0  |4  |5    |
+-----+------+-------+---+-----+



null

In [28]:
val times = lines.join(indexRun, "n").
    filter($"line".rlike("totalTime")).
    map{ m =>
        (m.getInt(2), m.getString(1).split("\t")(1))
    }.
    toDF("runID", "time")
times.show(20, truncate = false)

+-----+-----------+
|runID|time       |
+-----+-----------+
|0    |45590      |
|1    |60340      |
|2    |96550      |
|3    |160210     |
|4    |274740     |
|5    |531350     |
|6    |1.09247e+06|
|7    |5.68461e+06|
|8    |2.21501e+07|
+-----+-----------+



null

In [29]:
val data = runs.join(times, "runID")
data.show

+-----+------+-------+---+-----+-----------+
|runID|method|epsilon| mu|delta|       time|
+-----+------+-------+---+-----+-----------+
|    0|   bfe|   40.0|  4|    5|      45590|
|    1|   bfe|   50.0|  4|    5|      60340|
|    2|   bfe|   60.0|  4|    5|      96550|
|    3|   bfe|   70.0|  4|    5|     160210|
|    4|   bfe|   80.0|  4|    5|     274740|
|    5|   bfe|   90.0|  4|    5|     531350|
|    6|   bfe|  100.0|  4|    5|1.09247e+06|
|    7|   bfe|  150.0|  4|    5|5.68461e+06|
|    8|   bfe|  200.0|  4|    5|2.21501e+07|
+-----+------+-------+---+-----+-----------+



null

In [30]:
val d = data.collect.map(_.mkString(";")).mkString("\n")

import java.io._
val pw = new PrintWriter(new File(s"${folder}bfe_${prefix}.csv" ))
pw.write(s"$d\n")
pw.close

null

In [17]:
val sj_stages = stages.join(runs, "runID").
    filter($"method" === "SpatialJoin").
    select($"runID", $"n", $"method", $"epsilon", $"mu", $"delta", $"stage".alias("stage0"), $"stageTime").
    withColumn("stage1", regexp_replace($"stage0", "\\.\\.\\.", "")).
    withColumn("stage2", regexp_replace($"stage1", "Reporting", "0.Reporting")).
    select($"runID", $"n", $"method", $"epsilon", $"mu", $"delta", $"stage2".alias("stage"), $"stageTime").
    filter(!$"stage".rlike("4.Distance Join phase")).
    filter(!$"stage".rlike("5.Getting candidates"))
sj_stages.show(truncate = false)



+-----+---+------+-------+---+-----+-----+---------+
|runID|n  |method|epsilon|mu |delta|stage|stageTime|
+-----+---+------+-------+---+-----+-----+---------+
+-----+---+------+-------+---+-----+-----+---------+



null

In [18]:
val data = runs.select($"runID", $"method", $"epsilon", $"mu", $"delta", $"methodTime".alias("time")).
    filter($"method" === "MergeLast").filter($"mu" === 4 || $"mu" === 3).filter($"delta" === 3 || $"delta" === 5).
    orderBy($"runID", $"epsilon", $"method").
    cache
println(data.count())
data.show(data.count().toInt, truncate = false)

5
+-----+---------+-------+---+-----+-------+
|runID|method   |epsilon|mu |delta|time   |
+-----+---------+-------+---+-----+-------+
|0    |MergeLast|100.0  |4  |5    |927.875|
|1    |MergeLast|60.0   |4  |5    |654.964|
|2    |MergeLast|70.0   |4  |5    |686.385|
|3    |MergeLast|80.0   |4  |5    |677.862|
|4    |MergeLast|90.0   |4  |5    |798.868|
+-----+---------+-------+---+-----+-------+



null

In [19]:
val d = data.collect.map(_.mkString(";")).mkString("\n")

import java.io._
val pw = new PrintWriter(new File(s"${folder}methods_${prefix}.csv" ))
pw.write(s"$d\n")
pw.close

null

In [20]:
val d = ml_stages.union(sj_stages).collect.map(_.mkString(";")).mkString("\n")

import java.io._
val pw = new PrintWriter(new File(s"${folder}stages_${prefix}.csv" ))
pw.write(s"$d\n")
pw.close

null

In [ ]:
val indicesMdf = lines.filter{ l => 
        l.line.contains(" -> Setting mu=") || l.line.contains(" ->   berlin0-10,") 
    }
    .orderBy("n")
    .select("n")
    .collect()
    .toList
    .map(_.getLong(0))
    .grouped(2)
    .toList
    .map(pair => (pair.head, pair.last))
    .filter(r => r._1 != r._2)
    .zipWithIndex
val indexMdf = spark.createDataset(indicesMdf)
    .flatMap{ pair => 
        (pair._1._1 to pair._1._2)
        .toList.map(v => (pair._2, v))
    }
    .toDF("mdfID","n")
    .cache
indexMdf.show(15, truncate = false)

In [ ]:
val mdfInfo = indexMdf.groupBy($"mdfID").agg(max($"n").alias("m")).orderBy($"m")
mdfInfo.show(10, truncate = false)

In [ ]:
val mdfInfo = indexMdf.groupBy($"mdfID").agg(max($"n").alias("n")).orderBy($"n").
    join(indexRun, "n").
    join(runs, "runID").
    join(lines, "n").
    select($"method", $"epsilon", $"mu", $"delta", $"line").
    withColumn("timestamp", substring($"line", 122, 124)).
    select($"method", $"epsilon", $"mu", $"delta", $"timestamp")

mdfInfo.show(truncate = false)

In [ ]:
val d = mdfInfo.collect.map(_.mkString(";")).mkString("\n")

import java.io._
val pw = new PrintWriter(new File(s"${folder}mdfInfo_${prefix}.csv" ))
pw.write(s"$d\n")
pw.close